# Решение С.Ф. Рудницкого

In [8]:
#!pip install map-boxes

In [5]:
#!pip install pandas

In [6]:
#!pip install -U scikit-learn scipy matplotlib

In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random

In [2]:
#!git clone https://github.com/ultralytics/yolov5.git

In [3]:
#!pip install -r ./yolov5/requirements.txt

In [4]:
len(os.listdir('./participants/test/images'))

855

In [5]:
len(os.listdir('./participants/train/images'))

2002

In [6]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir('./participants/train/labels')]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [7]:
#!mkdir data_for_yolo
#!mkdir data_for_yolo/data

In [8]:
#!mkdir data_for_yolo/data/images
#!mkdir data_for_yolo/data/labels

In [ ]:
#!mkdir data_for_yolo/data/images/train
#!mkdir data_for_yolo/data/labels/train
#!mkdir data_for_yolo/data/images/test
#!mkdir data_for_yolo/data/labels/test

In [ ]:
#в папке data_for_yolo создадим dataset.yaml со следующим содержимым

yaml_content = """
train: /content/data_for_yolo/data/images/train/
val: /content/data_for_yolo/data/images/test/

# number of classes
nc: 5

# class names
names: ['car', 'head', 'face', 'human', 'carplate']
"""

#car': 0, 'head': 1, 'face': 2, 'human': 3, 'carplate': 4

In [ ]:
# Создаем датасет, который поможет преобразовать исходные данные для Yolo5

In [3]:
i = 0
data_labels = pd.DataFrame(columns=['ImageID', 'Label', 'XMin', 'XMax', 'YMin', 'YMax'])
for file in os.listdir('./participants/train/labels'):
    if '_car..txt' in file: 
        ImageID = file.replace('_car..txt', '')
        Label = 0
    elif '_head..txt' in file:     
        ImageID = file.replace('_head..txt', '')
        Label = 1
    elif '_face..txt' in file:         
        ImageID = file.replace('_face..txt', '')
        Label = 2
    elif '_human..txt' in file:
        ImageID = file.replace('_human..txt', '')
        Label = 3
    elif '_carplate..txt' in file:             
        ImageID = file.replace('_carplate..txt', '')
        Label = 4
    with open('./participants/train/labels/' + file, 'r') as f:
        data = f.read()
        data = [i for i in data.split('\n') if i != '']
    for line in data:
        val = [float(i) for i in line.split()]
        Label0, XMin, XMax, YMin, YMax = val
        new_row = {'ImageID': ImageID , 'Label':Label , 'XMin':XMin, 'XMax':XMax, 'YMin':YMin, 'YMax':YMax}
        data_labels = pd.concat([data_labels, pd.DataFrame(new_row, index=[0])], axis=0, ignore_index=True)        
        i += 1
        print(i, end='\r')

In [4]:
data_labels.head(40)

,ImageID,Label,XMin,XMax,YMin,YMax
0,0_23-11-2021_02-19-47_PM,4,0.819149,0.848485,0.152482,0.090909
1,0_23-11-2021_02-20-00_PM,4,0.176000,0.566298,0.126000,0.171271
2,0_23-11-2021_02-20-04_PM,4,0.124060,0.537931,0.093985,0.168965
3,0_23-11-2021_02-20-07_PM,4,0.759184,0.615764,0.163265,0.192118
4,0_23-11-2021_02-20-15_PM,4,0.190114,0.596774,0.150190,0.188172
5,0_23-11-2021_02-20-31_PM,4,0.345890,0.790503,0.270548,0.072626
6,0_23-11-2021_02-36-38_PM,4,0.143243,0.742991,0.135135,0.121495
7,0_23-11-2021_03-13-26_PM,4,0.387500,0.617647,0.231250,0.182353
8,0_23-11-2021_03-50-12_PM,4,0.236749,0.687500,0.173145,0.113636
9,0_23-11-2021_04-25-39_PM,4,0.714575,0.921725,0.196356,0.030351


In [5]:
images_names = data_labels['ImageID'].unique()

In [6]:
images_names[:20]

array(['0_23-11-2021_02-19-47_PM', '0_23-11-2021_02-20-00_PM',
       '0_23-11-2021_02-20-04_PM', '0_23-11-2021_02-20-07_PM',
       '0_23-11-2021_02-20-15_PM', '0_23-11-2021_02-20-31_PM',
       '0_23-11-2021_02-36-38_PM', '0_23-11-2021_03-13-26_PM',
       '0_23-11-2021_03-50-12_PM', '0_23-11-2021_04-25-39_PM',
       '0_23-11-2021_04-26-18_PM', '0_23-11-2021_05-01-40_PM',
       '0_23-11-2021_05-01-46_PM', '0_24-11-2021_01-13-52_PM',
       '0_24-11-2021_01-13-58_PM', '0_24-11-2021_01-38-00_AM',
       '0_24-11-2021_01-48-42_PM', '0_24-11-2021_01-48-49_PM',
       '0_24-11-2021_01-55-52_AM', '0_24-11-2021_01-56-17_AM'],
      dtype=object)

In [7]:
train_images_names, test_images_names = train_test_split(images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

1601 401


In [10]:
#train
for train_image in train_images_names:
    shutil.copy('./participants/train/images/' + train_image + '.jpg', './content/data_for_yolo/data/images/train/'+ train_image + '.jpg')    
    df = data_labels.loc[data_labels['ImageID']==train_image]
    with open('./content/data_for_yolo/data/labels/train/' + train_image + '.txt', 'w') as f:
        for i, row in df.iterrows():
            s = f"{row['Label']} {row['XMin']} {row['XMax']} {row['YMin']} {row['YMax']}"+"\n"
            f.write(s)
        f.close()

In [12]:
#test
for test_image in test_images_names:
    shutil.copy('./participants/train/images/' + test_image + '.jpg', './content/data_for_yolo/data/images/test/'+ test_image + '.jpg')    
    df = data_labels.loc[data_labels['ImageID']==test_image]
    with open('./content/data_for_yolo/data/labels/test/' + test_image + '.txt', 'w') as f:
        for i, row in df.iterrows():
            s = f"{row['Label']} {row['XMin']} {row['XMax']} {row['YMin']} {row['YMax']}"+"\n"
            f.write(s)
        f.close()

In [13]:
#валидация 
print(len(os.listdir('./content/data_for_yolo/data/images/test')))
print(len(os.listdir('./content/data_for_yolo/data/images/train')))

401
1601


In [14]:
print(len(os.listdir('./content/data_for_yolo/data/labels/test')))
print(len(os.listdir('./content/data_for_yolo/data/labels/train')))

401
1601


In [15]:
#обучение

In [48]:
path_to_data = './content/data_for_yolo/dataset.yaml'

In [49]:
#!nvidia-smi

In [50]:
#!python ./yolov5/train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg ./yolov5/models/yolov5s.yaml --name yolov5s_results --cache
!python ./yolov5/train.py --img 416 --batch 2 --epochs 1 --data {path_to_data} --cfg ./yolov5/models/yolov5s.yaml --name yolov5s_results --cache

detect: weights=['./yolov5/runs/train/yolov5s_results8/weights/best.pt'], source=./content/data_for_yolo/data/images/test, data=yolov5\data\coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-400-gfd004f5 Python-3.10.4 torch-1.12.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/401 O:\!P\!P00_AI_hack_ai\11\content\data_for_yolo\data\images\test\0_23-11-2021_02-19-47_PM.jpg: 320x416 Done. (0.097s)
image 2/401 O:\!P\!P00_AI_hack_ai\11\content\data_for_yolo\data\images\test\0_23-11-2021_02-20-31_PM.jpg: 416x352 Done. (0.102s)
image 3/401 O:\!P\!P00_AI_hack_ai\11\content\data_for_yolo\data\images\te

Command 'git fetch origin' timed out after 5 seconds


train: weights=yolov5\yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=./content/data_for_yolo/dataset.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=1, batch_size=2, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
From https://github.com/ultralytics/yolov5
 * [new branch]      update/Profile_class -> origin/update/Profile_class
YOLOv5  v6.1-400-gfd004f5 Python-3.10.4 torch-1.12.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5,

In [51]:
!python ./yolov5/detect.py --weights ./yolov5/runs/train/yolov5s_results9/weights/best.pt --img 416 --conf 0.4 --source ./content/data_for_yolo/data/images/test --save-txt --save-conf

detect: weights=['./yolov5/runs/train/yolov5s_results9/weights/best.pt'], source=./content/data_for_yolo/data/images/test, data=yolov5\data\coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-400-gfd004f5 Python-3.10.4 torch-1.12.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/401 O:\!P\!P00_AI_hack_ai\11\content\data_for_yolo\data\images\test\0_23-11-2021_02-19-47_PM.jpg: 320x416 Done. (0.122s)
image 2/401 O:\!P\!P00_AI_hack_ai\11\content\data_for_yolo\data\images\test\0_23-11-2021_02-20-31_PM.jpg: 416x352 Done. (0.149s)
image 3/401 O:\!P\!P00_AI_hack_ai\11\content\data_for_yolo\data\images\te

In [52]:
def get_soliton_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [53]:
simple_solution = get_soliton_labels_df('yolov5/runs/detect/exp/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [54]:
def get_test_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, center_x, center_y, width, height = val
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, xmin, xmax, ymin, ymax])
    return simple_solution

In [55]:
test_labels = get_test_labels_df('./content/data_for_yolo/data/labels/test/')
test_labels = pd.DataFrame(test_labels, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

In [56]:
mean_ap, average_precisions = mean_average_precision_for_boxes(test_labels, simple_solution, iou_threshold=0.5, verbose=False)

In [57]:
mean_ap

0.04526696973780063

In [58]:
#/content/participants/test/images

In [60]:
!python yolov5/detect.py --weights yolov5/runs/train/yolov5s_results9/weights/best.pt --img 416 --conf 0.4 --source ./participants/test/images --save-txt --save-conf

detect: weights=['yolov5/runs/train/yolov5s_results8/weights/best.pt'], source=./participants/test/images, data=yolov5\data\coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-400-gfd004f5 Python-3.10.4 torch-1.12.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 O:\!P\!P00_AI_hack_ai\11\participants\test\images\0_23-11-2021_02-18-50_PM.jpg: 288x416 2 cars, Done. (0.124s)
image 2/855 O:\!P\!P00_AI_hack_ai\11\participants\test\images\0_23-11-2021_02-18-56_PM.jpg: 352x416 3 cars, Done. (0.135s)
image 3/855 O:\!P\!P00_AI_hack_ai\11\participants\test\images\0_23-11-2021_02-19-19_PM.jpg: 256x416 D

detect: weights=['yolov5/runs/train/yolov5s_results9/weights/best.pt'], source=./participants/test/images, data=yolov5\data\coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-400-gfd004f5 Python-3.10.4 torch-1.12.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 O:\!P\!P00_AI_hack_ai\11\participants\test\images\0_23-11-2021_02-18-50_PM.jpg: 288x416 Done. (0.127s)
image 2/855 O:\!P\!P00_AI_hack_ai\11\participants\test\images\0_23-11-2021_02-18-56_PM.jpg: 352x416 Done. (0.136s)
image 3/855 O:\!P\!P00_AI_hack_ai\11\participants\test\images\0_23-11-2021_02-19-19_PM.jpg: 256x416 Done. (0.113s)
im

In [61]:
def get_soliton_labels_df_simple_solution(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            _, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, 3, conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [80]:
simple_solution = get_soliton_labels_df_simple_solution('yolov5/runs/detect/exp8/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
simple_solution

In [70]:
df = pd.read_csv("RSF_sample_solution.csv", sep=';', index_col=None)

In [71]:
df

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,0_24-11-2021_02-41-59_PM.jpg,3,0.425165,0.053192,0.175532,8.947360e-02,0.184210
1,0_26-11-2021_10-11-18_AM.jpg,3,0.434180,0.278689,0.346311,1.865690e-02,0.130597
2,23-11-2021_02-18-50_PM.jpg,3,0.557728,0.667708,0.783854,2.481485e-01,0.379629
3,23-11-2021_02-18-50_PM.jpg,3,0.638038,0.838542,0.994270,5.009260e-01,0.800000
4,23-11-2021_02-18-50_PM.jpg,3,0.707852,0.582292,0.706250,2.759255e-01,0.394444
...,...,...,...,...,...,...,...
4399,26-11-2021_12-47-01_PM.jpg,3,0.709888,0.288542,0.419270,5.037040e-01,0.657408
4400,26-11-2021_12-47-01_PM.jpg,3,0.714639,0.511459,0.703126,5.981485e-01,0.829630
4401,26-11-2021_12-47-01_PM.jpg,3,0.722404,0.055208,0.123438,5.620370e-01,0.710185
4402,2_24-11-2021_10-14-19_AM.jpg,3,0.405010,0.921569,0.994117,0.000000e+00,0.067669


In [73]:
RSF_simple_solution = get_soliton_labels_df_simple_solution('yolov5/runs/detect/exp8/labels/')
RSF_simple_solution

[['0_24-11-2021_02-41-59_PM.jpg',
  3,
  0.425165,
  0.053192,
  0.17553200000000002,
  0.08947359999999999,
  0.1842104],
 ['0_26-11-2021_10-11-18_AM.jpg',
  3,
  0.43418,
  0.2786885,
  0.3463115,
  0.018656899999999997,
  0.1305969],
 ['23-11-2021_02-18-50_PM.jpg',
  3,
  0.557728,
  0.667708,
  0.783854,
  0.2481485,
  0.37962949999999995],
 ['23-11-2021_02-18-50_PM.jpg',
  3,
  0.638038,
  0.8385415,
  0.9942705000000001,
  0.500926,
  0.8],
 ['23-11-2021_02-18-50_PM.jpg',
  3,
  0.707852,
  0.582292,
  0.70625,
  0.2759255,
  0.39444450000000003],
 ['23-11-2021_02-18-50_PM.jpg',
  3,
  0.748325,
  0.6645840000000001,
  0.832292,
  0.4648145,
  0.6981475],
 ['23-11-2021_02-18-58_PM.jpg',
  3,
  0.517016,
  0.9343745999999999,
  0.9786454,
  0.390741,
  0.641667],
 ['23-11-2021_02-18-58_PM.jpg',
  3,
  0.696473,
  0.91458315,
  0.9968748500000001,
  0.448148,
  0.593518],
 ['23-11-2021_02-19-04_PM.jpg',
  3,
  0.587528,
  0.629688,
  0.816146,
  0.531482,
  0.853704],
 ['23-11-2021

In [81]:
simple_solution.to_csv("RSF_simple_solution.csv", sep=';', index=False)

In [76]:
RSF_simple_solution = pd.DataFrame(RSF_simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [78]:
RSF_simple_solution = RSF_simple_solution.drop('Conf', axis=1) 

In [79]:
RSF_simple_solution.to_csv("RSF_sample_solution.csv", sep=';', index=False)